# Manuscript

Finally time to bring it all together and make a manuscript with all the details and dialog!

We'll use the power of llamaindex to maintain coherence and manage the context window.

:warning: TODO: Use a different writer for each medium type

In [ ]:
# Can Render to a PDF using pandoc
!sudo apt -y install pandoc
%pip install markdown2 pypandoc

# If using pdflatex
!sudo apt -y install texlive texlive-latex-extra

# Load the story outline

In [ ]:
import settings
from model import Story

story = Story.load_from_directory(settings.STORY_DIR + "/step_4")

story.display()

# Generate Chapters (Book)

In [ ]:
from model import Story, Act, Scene, Character

def generate_story_overview(story: Story) -> str:
    """Generates a Markdown overview for the story, including title, genre, setting, themes, and other high-level details to feed into the LLM."""

    markdown = f"# {story.title}\n\n"
    markdown += f"**Medium:** {story.medium}\n\n"
    markdown += f"**Genre:** {story.genre}\n\n"
    markdown += f"**Visual Style:** {story.visual_style}\n\n"
    markdown += f"**Time Period:** {story.time_period}\n\n"
    markdown += f"**Location:** {story.location}\n\n"
    markdown += f"**Narrative Perspective:** {story.narrative_perspective}\n\n"
    markdown += f"**Conflict Type:** {story.conflict_type}\n\n"
    
    # Themes and motifs
    if story.themes:
        markdown += f"**Themes:** {', '.join(story.themes)}\n\n"
    if story.motifs:
        markdown += f"**Motifs:** {', '.join(story.motifs)}\n\n"

    # Story Beats
    if story.story_beats:
        markdown += "## Story Beats\n\n"
        for beat in story.story_beats:
            markdown += f"- **{beat.name}:** {beat.description} (Scene: {beat.scene})\n"
        markdown += "\n"

    # Subplots
    if story.subplots:
        markdown += "## Subplots\n\n"
        for subplot in story.subplots:
            markdown += f"### Subplot: \"{subplot.title}\"\n"
            markdown += f"#### Description:\n{subplot.description}\n\n"
            markdown += f"#### Related Characters (nicknames):\n{', '.join(subplot.related_characters)}\n\n"
        markdown += "\n"

    # Emotional Arc
    if story.emotional_arc:
        markdown += "## Emotional Arc\n\n"
        for arc in story.emotional_arc:
            markdown += f"- **{arc.stage}:** {arc.description}\n"
        markdown += "\n"

    markdown += f"## Original User Prompt:\n{story.prompt}\n\n"
    markdown += f"## Plot Overview:\n{story.plot_overview}\n\n"
    return markdown

def generate_character_summary(character: Character) -> str:
    """Generates a Markdown summary for a single character, including role, description, and internal conflict."""
    markdown = f"### Character: \"{character.name}\"\n\n"
    markdown += f"#### Role:\n{character.role}\n\n"
    markdown += f"#### Description:\n{character.description}\n\n"
    markdown += f"#### Personality:\n{character.personality}\n\n"
    markdown += f"#### Appearance:\n{character.physical_appearance}\n\n"
    markdown += f"#### Internal Conflict:\n{character.internal_conflict or "none"}\n\n"
    if character.character_arc:
        markdown += f"#### Character Arc:\n"
        markdown += f"- Initial State: {character.character_arc.initial_state}\n"
        markdown += f"- Final State: {character.character_arc.final_state}\n"
        markdown += f"- Key Moments: {', '.join(character.character_arc.key_moments)}\n\n"
    return markdown

def generate_act_summary(act: Act, include_scenes: bool = True) -> str:
    """Generates a Markdown summary for an act, optionally including details on each scene."""
    markdown = f'### Act: "{act.title}"\n\n'
    markdown += f"**Description:**\n{act.description}\n\n"

    if include_scenes:
        for scene in act.scenes:
            markdown += generate_scene_summary(scene)
    return markdown

def generate_scene_summary(scene: Scene) -> str:
    """Generates a Markdown summary for a single scene, including setting, characters involved, and key actions."""
    markdown = f"#### Scene: \"{scene.title}\"\n\n"
    markdown += f"**Scene ID:** {scene.scene_id}\n"
    markdown += f"**Setting:** {scene.setting}\n"
    markdown += f"**Time of Day:** {scene.time_of_day}\n"
    markdown += f"**Location:** {scene.location}\n"
    markdown += f"**Lighting:** {scene.lighting}\n"
    markdown += f"**Mood:** {scene.mood}\n"
    markdown += f"**Characters Involved (by nickname):** {', '.join(scene.characters_involved)}\n"
    markdown += f"**Props:** {', '.join(scene.props)}\n"

    if scene.key_actions:
        markdown += f"##### Key Actions:\n"
        for action in scene.key_actions:
            markdown += f"- {action}\n"
        markdown += "\n"

    markdown += f"##### Description:\n{scene.description}\n\n"

    return markdown

def generate_full_story_summary(story: Story, include_characters: bool = True, include_acts: bool = True, include_scenes: bool = True) -> str:
    """Generates a full Markdown summary for the entire story, including overview, characters, acts, and scenes."""
    markdown = generate_story_overview(story)
    
    # Characters
    if include_characters:
        markdown += "\n\n---\n\n## Characters\n\n"
        for character in story.characters:
            markdown += generate_character_summary(character) + "\n"
    
    # Acts and Scenes
    if include_acts:
        markdown += "\n\n---\n\n## Acts and Scenes\n\n"
        for act in story.acts:
            markdown += generate_act_summary(act, include_scenes=include_scenes) + "\n"

    return markdown


In [ ]:
from IPython.display import Markdown, display
import ipywidgets as widgets
from model_text import display_messages
from llama_index.core.llms import ChatMessage
from model_text import llm, stream_llm_response
from utils import blank_story, blank_story_dialog, deindent
from model import Story, DialogueLine, ActDialogue, SceneDialogue, StoryDialogue, ActDialogue, SceneDialogue, DialogueLine

# Define exclusions for the user prompt and LLM response
request_excludes = {
    "characters": {
        "__all__": {
            "animation_description": True,
            "voice_description": True,
            # "internal_conflict": True,
            # "character_arc": True,
            "voice_sample": True,
            "catch_phrase": True,
            "props": True,
            "image_prompt": True,
            "image_prompt_short": True,
        }
    }, 
    "acts": {
        "__all__": {
            "scenes": {
                "__all__": {
                    "scene_image_prompt": True,
                    "scene_image_prompt_short": True,
                    "background_animation": True,
                    "background_image_prompt": True,
                    "dialogue": True,
                }
            }
        }
    },
}

def generate_manuscript(story: Story) -> StoryDialogue:
    total_scenes = sum([len(act.scenes) for act in story.acts])
    approx_chunks = 500 # Just to show progress as a chunks come in
    progress = widgets.IntProgress(value=0, min=0, max=total_scenes * approx_chunks)
    scene_number = 0
    display(progress)

    # Initialize StoryDialogue
    story_dialogue = story.get_story_dialogue()
    if not story_dialogue:
        story_dialogue = StoryDialogue()
    story.set_story_dialogue(story_dialogue)

    story_context = generate_full_story_summary(story)

    # TODO: Support for narrator/dialogue distinction

    display(Markdown(f"### Generating Story: {story.title}"))

    for act in story.acts:
        display(Markdown(f"#### Generating Act: {act.title}"))
        act_dialogue = ActDialogue(act_id=act.act_id)
        story_dialogue.act_dialogues.append(act_dialogue)

        for scene in act.scenes:
            progress.value = scene_number * approx_chunks
            scene_number += 1
        
            display(Markdown(f"##### Generating Scene: {scene.title}"))

            scene_dialogue = SceneDialogue(scene_id=scene.scene_id)
            act_dialogue.scene_dialogues.append(scene_dialogue)

            system_message = deindent(f"""
                You are an exceptionally talented and creative book author, specializing in crafting vivid, narrative-driven chapters for a {story.medium}.
                
                Write a complete chapter in natural, printable prose based on the scene provided. The output should be **only the narrative content of the chapter** as if it were ready for publication. 

                Guidelines:
                - No headers, titles, or meta information: Do not add any headers, titles, genre notes, or extra explanations.
                - Only narrative prose: Write as if this is a final printed chapter in a novel, without character labels or any structural cues.
                - Make it immersive: Use rich descriptions of settings, character actions, thoughts, and dialogue to move through the scene, and focus on sensory details to make the reader feel present.
                - Pacing: Fully develop each part of the scene, including character interactions, conflicts, emotions, and subtleties. Take your time with the narrative, allowing it to unfold naturally and build tension or depth.

                Suggested Structure:
                - **Begin** with an immersive setting description to establish time, place, and atmosphere.
                - **Progress** through the scene using detailed character actions, interactions, thoughts, and emotions, taking the time to explore each element.
                - **Conclude** naturally, allowing for smooth continuity into the following chapter if applicable.

                Do not output anything but a completed chapter in markdown format.
                Only output ASCII characters. Do not include any special characters or emojis like smark quotes and em dashes.

                DO NOT OUTPUT ANYTHING BUT A COMPLETED CHAPTER!

                Output in markdown. Do not include chapter titles.

                Here is the overview of the story for context: 
            """)
            # TODO: adjust the chapter length according to the medium
            system_message += "\n\n" + story_context
            # print(system_message)
            messages = [ChatMessage(role="system", content=system_message)]
            user_prompt = deindent(f"""
                Write an expanded, full-length chapter for the scene "{scene.title}" in the act "{act.title}" in markdown format. 
                Focus on creating a fully immersive experience, as though it is a complete chapter in a novel. Do not include the title or any meta information.
            """)

            messages.append(ChatMessage(role="user", content=user_prompt))
            
            # Debug display of messages
            display_messages(messages)

            # Generate dialogue using llm_json
            # response = llm.chat(messages, max_new_tokens=100000).message.content
            response = llm.stream_chat(messages, max_new_tokens=100000)
            scene_dialogue.content = stream_llm_response(response, progress=progress)

            # display(Markdown(f"### Scene Content: {scene_dialogue.content}"))

            # if settings.DEBUG:
            # display(Markdown(f"```\n{response}\n```"))

            story.save_to_directory(settings.STORY_DIR + "/step_14")
            display(Markdown(f":white_check_mark: Saved story progress!"))


    return story_dialogue


## Generate the book manuscript

In [ ]:
manuscript = generate_manuscript(story)

IntProgress(value=0, max=3000)

### Generating Story: Beyond the Pixel Veil

#### Generating Act: Act 1: Beyond the Screen

##### Generating Scene: The Underdog's Rise

The air was electric with anticipation as Meera 'Midnight' Singh stepped onto the vibrant, neon-lit stage of New Mumbai's
 Cyber Arena. The crowd's deafening roar enveloped her, a cacophony of cheers and chants that threatened to consume her entire
 being. With a deep breath, she slipped on her trusty headset, the familiar weight a comforting reminder of the battles she
 was about to face.

As she gazed out into the sea of expectant faces, Meera's thoughts drifted back to the countless hours spent honing her craft
 in cramped, dimly lit internet cafes. The underdog from the outskirts of New Mumbai had finally made it to the big leagues
 – the Eon Gaming Tournament's grand finale. Her team, 'Midnight's Marauders,' was set to take on the reigning champions,
 'Specter's Squad,' led by the charismatic Jax 'Specter' Lee.

Meera's eyes locked onto her teammates, each one a vital cog in their well-oiled machine. There was Rohan 'Brawler' Jensen
, their tanky powerhouse; Leela 'Lumina' Rao, th

...

The darkness was absolute, punctuated only by the faint hum of emergency generators and the soft murmur of hushed conversations
. Meera's eyes, accustomed to the vibrant neon hues of New Mumbai's cityscape, struggled to adjust to the dimly lit landscape
 before her. The air reeked of smoke, ozone, and desperation.

As she stepped out of the makeshift medical tent, the chill of the night air enveloped her, carrying with it the whispers
 of the devastated. Meera's gaze wandered across the sea of makeshift shelters, the rubble-strewn streets, and the twisted
 wreckage of what once were homes. The weight of her actions in the Nexus crushed her, threatening to consume her very being
.

"Meera, we need to talk," a low, urgent voice called out from beside her.

She turned to face Kaito 'Sparrow' Hernandez, leader of the underground rebellion against the corporations exploiting Eon
. His bright hazel eyes, usually ablaze with determination, now seemed dulled by the exhaustion and grief etched 

:white_check_mark: Saved story progress!

##### Generating Scene: Confronting the Architect

The air was alive with the pulse of distorted code, each step echoing through the Architect's Domain like a challenge to the
 very fabric of reality. Meera's heart raced in tandem, her senses on high alert as she led her team deeper into the ever
-shifting labyrinth. Jax flanked her left, his eyes scanning the surroundings with a mixture of fascination and wariness,
 while Dr. Zhang and Maya brought up the rear, their faces set in determined lines.

As they turned a corner, the landscape before them dissolved into a sea of glitch-art constructs, each one a mesmerizing dance
 of light and color. The team's footsteps slowed, awestruck by the sheer scale of the display. Meera, however, pressed on
, her gaze fixed on the figure waiting at the far end of the shimmering expanse.

The Architect, Erebus, stood poised before a backdrop of churning code, its androgynous form shifting subtly with each passing
 moment. The being's presence was both captivating and unnerving, like gazing into the h

:white_check_mark: Saved story progress!

## Setup llamaindex

In [ ]:
# %pip install llama-index-core

# if settings.TEXT_MODEL_BACKEND == "ollama":
#     %pip install llama-index-embeddings-ollama

In [ ]:
# from IPython.display import Markdown, display
# from model_text import display_messages
# from llama_index.core.llms import ChatMessage
# from utils import blank_story, deindent
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# from model_text import llm, llm_json, embedding

# # Create an empty index to store generated content
# index = VectorStoreIndex([], embed_model=embedding)
# query_engine = index.as_query_engine(llm=llm)

In [ ]:
# # Index the story content for context
# from llama_index.core import Document

# def story_to_documents(story):
#     documents = []
    
#     # Add overall story information
#     documents.append(Document(text=f"Title: {story.title}\nGenre: {story.genre}\nMedium: {story.medium}\nVisual Style: {story.visual_style}\nTime Period: {story.time_period}\nLocation: {story.location}\nPlot Overview: {story.plot_overview}\nNarrative Perspective: {story.narrative_perspective}"))
    
#     # Add characters
#     for character in story.characters:
#         char_doc = f"Character: {character.name}\nRole: {character.role}\nDescription: {character.description}\nPersonality: {character.personality}\nPhysical Appearance: {character.physical_appearance}\nCatch Phrase: {character.catch_phrase}\nInternal Conflict: {character.internal_conflict}"
#         documents.append(Document(text=char_doc))
    
#     # Add acts and scenes
#     for act in story.acts:
#         act_doc = f"Act: {act.title}\nDescription: {act.description}"
#         documents.append(Document(text=act_doc))
#         for scene in act.scenes:
#             scene_doc = f"Scene: {scene.title}\nDescription: {scene.description}\nSetting: {scene.setting}\nMood: {scene.mood}"
#             documents.append(Document(text=scene_doc))
    
#     # Add story beats
#     for beat in story.story_beats:
#         beat_doc = f"Story Beat: {beat.name}\nDescription: {beat.description}\nScene: {beat.scene}"
#         documents.append(Document(text=beat_doc))
    
#     # Add subplots
#     for subplot in story.subplots:
#         subplot_doc = f"Subplot: {subplot.title}\nDescription: {subplot.description}"
#         documents.append(Document(text=subplot_doc))
    
#     # Add emotional arc
#     for arc in story.emotional_arc:
#         arc_doc = f"Emotional Arc Stage: {arc.stage}\nDescription: {arc.description}"
#         documents.append(Document(text=arc_doc))
    
#     # Add themes and motifs
#     themes_doc = f"Themes: {', '.join(story.themes)}\nMotifs: {', '.join(story.motifs)}\nConflict Type: {story.conflict_type}"
#     documents.append(Document(text=themes_doc))
    
#     return documents

In [ ]:
# docs = story_to_documents(story)
# for doc in docs:
#     index.insert_nodes([doc])

In [ ]:
# from model import StoryDialog, ActDialog, SceneDialog, DialogueLine

# def generate_dialogue_line(scene_context):
#     response = query_engine.query(f"Generate a dialogue line for the scene: {scene_context}")
#     character, text = response.response.split(": ", 1)
#     return DialogueLine(character=character.strip(), text=text.strip())

# def generate_scene(act_context):
#     response = query_engine.query(f"Generate a scene description for the act: {act_context}")
#     scene = SceneDialog(scene_description=response.response, dialogues=[])
    
#     for _ in range(3):  # Generate 3 dialogue lines per scene
#         scene.dialogues.append(generate_dialogue_line(scene.scene_description))
    
#     return scene

# def generate_act(story_context):
#     response = query_engine.query(f"Generate an act description for the story: {story_context}")
#     act = ActDialog(act_description=response.response, scenes=[])
    
#     for _ in range(3):  # Generate 3 scenes per act
#         act.scenes.append(generate_scene(act.act_description))
    
#     return act

# def generate_story(title):
#     story = StoryDialog(title=title, acts=[])
    
#     for _ in range(3):  # Generate 3 acts
#         act = generate_act(story.title)
#         story.acts.append(act)
        
#         # Add generated content to the index for context
#         index.insert_nodes([act.act_description])
#         for scene in act.scenes:
#             index.insert_nodes([scene.scene_description])
#             for dialogue in scene.dialogues:
#                 index.insert_nodes([f"{dialogue.character}: {dialogue.text}"])
    
#     return story

In [ ]:
# def generate_complete_manuscript(story):
#     manuscript = StoryDialog(act_dialogues=[])
    
#     for act_index, act in enumerate(story.acts):
#         act_dialog = ActDialog(act_id=act.act_id, scene_dialogues=[])
        
#         for scene_index, scene in enumerate(act.scenes):
#             scene_dialog = SceneDialog(scene_id=scene.scene_id, dialogues=[])
#             last_dialogue_line = ""
            
#             while True:
#                 response = query_engine.query(
#                     f"Continue the dialogue for scene '{scene.description}'. "
#                     f"Previous dialogue: '{last_dialogue_line}'. "
#                     f"Generate the next dialogue line using the format 'Character: Dialogue'. "
#                     f"Ensure you use a colon (:) to separate the character name from their dialogue. "
#                     f"If the scene is complete, end your response with 'Scene complete'."
#                 )
                
#                 if "Scene complete" in response.response or len(scene_dialog.dialogues) >= 10:
#                     print(f"Scene '{scene.description}' generation complete.")
#                     break
                
#                 try:
#                     character, text = response.response.split(": ", 1)
#                 except ValueError:
#                     print("Unexpected format in dialogue line. Skipping.")
#                     continue
                
#                 dialogue_line = DialogueLine(character_nickname=character.strip(), line=text.strip())
#                 scene_dialog.dialogues.append(dialogue_line)
                
#                 last_dialogue_line = f"{character}: {text}"
                
#                 index.insert_nodes([f"{character}: {text}"])

#             act_dialog.scene_dialogues.append(scene_dialog)
        
#         manuscript.act_dialogues.append(act_dialog)
    
#     return manuscript

In [ ]:
# manuscript = generate_complete_manuscript(story)

In [ ]:
# dir(index)
# index.summary

# Convert to Markdown and LaTeX

## Load the manuscript

In [3]:
import settings
from model import Story

story = Story.load_from_directory(settings.STORY_DIR + "/step_14")

In [4]:
def story_to_markdown(story):
    """
    Converts a Story and its associated StoryDialog to a Markdown manuscript, formatted like a novel with chapter pages.
    """
    # Start with the title page
    markdown_content = f"# {story.title}\n\n"
    
    # Loop over each act and scene to create the story content
    for act_index, (act, act_dialog) in enumerate(zip(story.acts, story.get_story_dialogue().act_dialogues)):
        # Add a chapter title page for each act
        markdown_content += f"\n\n# Chapter {act_index + 1}\n\n"
        markdown_content += f"_{act.description}_\n\n"  # Italicize the chapter description for a thematic touch
        markdown_content += "<div style='page-break-after: always;'></div>\n\n"  # Page break for chapter separation
        
        # Loop over each scene in the act
        for scene_index, (scene, scene_dialog) in enumerate(zip(act.scenes, act_dialog.scene_dialogues)):
            markdown_content += f"\n\n---\n\n"  # Separator for scenes
            
            # Optional: Add scene description for context
            markdown_content += f"_{scene.description}_\n\n" if scene.description else ""
            
            # Add each dialogue line for the scene
            for dialogue_line in scene_dialog.dialogues:
                # Dialogue formatted as prose
                markdown_content += f"**{dialogue_line.character_nickname}**: {dialogue_line.line}\n\n"
    
    return markdown_content


In [ ]:
# Render the markdown
markdown_content = story_to_markdown(story)

In [ ]:
# Save it
import os
os.makedirs(f"{settings.STORY_DIR}/step_14", exist_ok=True)
manuscript_md_path = f"{settings.STORY_DIR}/step_14/manuscript.md"

# Save to a Markdown file
with open(manuscript_md_path, "w") as file:
    file.write(markdown_content)

In [ ]:
import markdown2
import pypandoc

def markdown_to_latex(markdown_text):
    """
    Convert markdown text to LaTeX using markdown2 and pypandoc.
    """
    # Convert markdown to HTML first
    html_text = markdown2.markdown(markdown_text)
    # Convert HTML to LaTeX
    latex_text = pypandoc.convert_text(html_text, 'latex', format='html')
    return latex_text

In [89]:
def story_to_latex_graphic_novel(story, scene_image_dir):
    """
    Converts a Story and its associated StoryDialog to a LaTeX manuscript formatted as a graphic novel,
    where each scene is treated as a chapter and may include an image.
    """

    # Start with document preamble
    latex_content = r"""
    \documentclass[12pt]{report}  % Using 'report' class to avoid blank pages between chapters
    \usepackage{setspace}
    \usepackage{titlesec}
    \usepackage{graphicx}
    \usepackage{url}
    \titleformat{\chapter}[display]
      {\normalfont\huge\bfseries}{\chaptername\ \thechapter}{20pt}{\Huge}
    \usepackage{fancyhdr}
    \pagestyle{fancy}
    \fancyhf{}
    \rhead{\thepage}
    \begin{document}
    \onehalfspacing
    \pagenumbering{gobble} % Suppress page numbers until Chapter 1
    """

    # Title page with title image if available, without a page break
    title_image_path = os.path.join(scene_image_dir, "title.png")
    latex_content += r"\begin{center}\n"
    
    # Include the title image if available
    if os.path.exists(title_image_path):
        latex_content += f"\\includegraphics[width=0.8\\textwidth]{{{title_image_path}}}\n\\vspace{{1cm}}\n"
    
    # Add the title text below the image
    latex_content += r"""
    {\Huge \textbf{""" + story.title + r"""}} \\[2cm]
    \end{center}
    """

    # Start page numbering with Chapter 1
    latex_content += r"""
    \newpage
    \pagenumbering{arabic} % Start page numbering
    """

    # Loop over each scene to create the story content
    chapter_counter = 1
    for act, act_dialogue in zip(story.acts, story.get_story_dialogue().act_dialogues):
        for scene, scene_dialogue in zip(act.scenes, act_dialogue.scene_dialogues):
            # Each scene is a chapter
            latex_content += f"\\chapter*{{Chapter {chapter_counter}: {scene.title}}}\n"
            latex_content += f"\\addcontentsline{{toc}}{{chapter}}{{Chapter {chapter_counter}: {scene.title}}}\n"
            chapter_counter += 1

            # Optional scene description in italics
            if scene.description:
                latex_content += f"\\textit{{{scene.description}}}\n\n"
            
            # Include scene image if it exists
            scene_image_path = os.path.join(scene_image_dir, f"{scene.scene_id}.live.png")
            if os.path.exists(scene_image_path):
                latex_content += f"\\begin{{center}}\n\\includegraphics[width=0.8\\textwidth]{{{scene_image_path}}}\n\\end{{center}}\n\n"

            # Use the content field for the full scene dialogue and actions, converting from markdown to latex
            if scene_dialogue.content:
                latex_scene_content = markdown_to_latex(scene_dialogue.content)
                latex_content += f"{latex_scene_content}\n\n"

    # Add "Made with Plotomatic" statement at the end
    latex_content += r"""
    \newpage
    \vfill
    \begin{center}
    \textit{Made with \textbf{Plotomatic}.} \\
    For more information, visit: \url{https://github.com/mattwilliamson/Plotomatic}
    \end{center}
    """

    # End the document
    latex_content += "\\end{document}"
    
    return latex_content


In [90]:
# latex_content = story_to_latex_novel(story)
latex_content = story_to_latex_graphic_novel(story, settings.STORY_DIR + "/step_6/scenes")
latex_content

'\n    \\documentclass[12pt]{report}  % Using \'report\' class to avoid blank pages between chapters\n    \\usepackage{setspace}\n    \\usepackage{titlesec}\n    \\usepackage{graphicx}\n    \\usepackage{url}\n    \\titleformat{\\chapter}[display]\n      {\\normalfont\\huge\\bfseries}{\\chaptername\\ \\thechapter}{20pt}{\\Huge}\n    \\usepackage{fancyhdr}\n    \\pagestyle{fancy}\n    \\fancyhf{}\n    \\rhead{\\thepage}\n    \\begin{document}\n    \\onehalfspacing\n    \\pagenumbering{gobble} % Suppress page numbers until Chapter 1\n    \\begin{center}\\n\\includegraphics[width=0.8\\textwidth]{stories/my_story/step_6/scenes/title.png}\n\\vspace{1cm}\n\n    {\\Huge \\textbf{Beyond the Pixel Veil}} \\\\[2cm]\n    \\end{center}\n    \n    \\newpage\n    \\pagenumbering{arabic} % Start page numbering\n    \\chapter*{Chapter 1: The Underdog\'s Rise}\n\\addcontentsline{toc}{chapter}{Chapter 1: The Underdog\'s Rise}\n\\textit{Meera participates in a high-stakes Eon gaming tournament, showcasing

In [91]:
from IPython.display import display, Markdown

step_path = f"{settings.STORY_DIR}/step_14"
manuscript_json_path = f"{step_path}/manuscript.json"
manuscript_latex_path = f"{step_path}/manuscript.tex"
manuscript_pdf_path = f"{step_path}/manuscript.pdf"

# Save to a LaTeX file
with open(manuscript_latex_path, "w") as file:
    file.write(latex_content)

# !pandoc {manuscript_latex_path} -o {manuscript_pdf_path}
!ls -lah {step_path}
!rm manuscript.log
!pdflatex -interaction=nonstopmode -output-directory={step_path} {manuscript_latex_path}

total 4.2M
drwxrwxr-x  2 matt matt 4.0K Nov 12 14:41 .
drwxrwxr-x 15 matt matt 4.0K Nov 12 14:26 ..
-rw-rw-r--  1 matt matt  658 Nov 12 14:56 manuscript.aux
-rw-rw-r--  1 matt matt  15K Nov 12 14:56 manuscript.log
-rw-rw-r--  1 matt matt 1.2K Nov 12 14:00 manuscript.md
-rw-rw-r--  1 matt matt 4.1M Nov 12 14:56 manuscript.pdf
-rw-rw-r--  1 matt matt  28K Nov 12 14:58 manuscript.tex
-rw-rw-r--  1 matt matt  26K Nov 12 13:20 story_dialog.json
-rw-rw-r--  1 matt matt  35K Nov 12 13:02 story.json
rm: cannot remove 'manuscript.log': No such file or directory
This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./stories/my_story/step_14/manuscript.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/report.cls
Document Class: report 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/

In [92]:
display(Markdown(f"# :open_book: [Download the manuscript PDF]({manuscript_pdf_path})"))

# :open_book: [Download the manuscript PDF](stories/my_story/step_14/manuscript.pdf)

# Generate Dialog (for screenplays)
